## # Introduction
<p><img src="https://i.imgur.com/kjWF1So.jpg" alt="Different characters on a computer screen"></p>
<p>According to a 2019 <a href="https://storage.googleapis.com/gweb-uniblog-publish-prod/documents/PasswordCheckup-HarrisPoll-InfographicFINAL.pdf">Google / Harris Poll</a>, 24% of Americans have used common passwords, like <code>abc123</code>, <code>Password</code>, and <code>Admin</code>. Even more concerning, 59% of Americans have incorporated personal information, such as their name or birthday, into their password. This makes it unsurprising that 4 in 10 Americans have had their personal information compromised online. Passwords with commonly used phrases and personal information makes cracking a password drastically easier.</p>
<p>You may have noticed over the years that password requirements have increased in complexity, including recommendations to change your passwords every couple of months. Compiled from industry recommendations, below is a list of passwords requirements you will be asked to test: </p>
<p><strong>Password Requirments:</strong></p>
<ol>
<li>Must be at least 10 characters in length</li>
<li>Must contain at least:<ul>
<li>one lower case letter </li>
<li>one upper case letter </li>
<li>one numeric character </li>
<li>one non-alphanumeric character</li></ul></li>
<li>Must not contain the phrase <code>password</code> (case insensitive)</li>
<li>Must not contain the user's first or last name, e.g., if the user's name is <code>John Smith</code>, then <code>SmItH876!</code> is not a valid password.</li>
</ol>
<p>Here is the dataset that you will investigate this project:</p>
<div style="background-color: #ebf4f7; color: #595959; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
    <div style="font-size:20px"><b>datasets/logins.csv</b></div>
Each row represents a login credential. There are no missing values and you can consider the dataset "clean".
<ul>
    <li><b>id:</b> the user's unique ID.</li>
    <li><b>username:</b> the username with the format {firstname}.{lastname}.</li>
    <li><b>password:</b> the password that may or may not meet the requirements. <i>Note, passwords should never be saved in plaintext, always encrypt them when working with real live passwords!</i></li>
</ul>
</div>
<p>Warning: This dataset contains some <strong>real</strong> passwords leaked from <strong>real</strong> websites. These passwords have been filtered, but may still include words that are explicit and offensive.</p>
<p>From here on out, it will be your task to explore and manipulate the existing data until you can answer the two questions described in the instructions panel. Feel free to import as many packages as you need to complete your task, and add cells as necessary. Finally, remember that you are only tested on your answer, not on the methods you use to arrive at the answer!</p>
<p><strong>Note:</strong> To complete this project, you need to know how to manipulate strings in pandas DataFrames and be familiar with regular expressions. Before starting this project we recommend that you have completed the following courses: <a href="https://learn.datacamp.com/courses/data-cleaning-in-python">Data Cleaning in Python</a> and <a href="https://learn.datacamp.com/courses/regular-expressions-in-python">Regular Expressions in Python</a>.</p>

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('datasets/logins.csv')

In [4]:
# 1.Must be at least 10 characters in length
length_check = df["password"].str.len() >= 10
length_check.head()

0     True
1     True
2    False
3    False
4     True
Name: password, dtype: bool

In [5]:
valid_pw = df[length_check]
bad_pw = df[~length_check]

In [6]:
valid_pw.shape[0], bad_pw.shape[0]

(560, 422)

In [7]:
# 2.Must contain at least:one lower, one upper, one numeric, one non-alphanumeric character
lcase = valid_pw['password'].str.contains('[a-z]')
ucase = valid_pw['password'].str.contains('[A-Z]')
ncase = valid_pw['password'].str.contains('[\d]')
scase = valid_pw['password'].str.contains('[\W]')

In [8]:
pd.concat([valid_pw, lcase, ucase, ncase, scase], axis=1).head()

,id,username,password,password,password,password,password
0,1,vance.jennings,vanceRules888!,True,True,True,True
1,2,consuelo.eaton,Mail_Pen%Scarlets.414,True,True,True,True
4,5,araceli.wilder,Araceli}r3,True,True,True,True
5,6,shawn.harrington,126_239_123,False,False,True,False
6,7,evelyn.gay,`4:&iAt$'o~(,True,True,True,True


In [9]:
case_check = lcase & ucase & ncase & scase

bad_pw = bad_pw.append(valid_pw[~case_check], ignore_index=True)
valid_pw = valid_pw[case_check]

valid_pw.shape[0], bad_pw.shape[0]

(258, 724)

In [10]:
# 3.Must not contain the phrase password (case insensitive)
not_valid = valid_pw["password"].str.contains("password", case=False)

bad_pw = bad_pw.append(valid_pw[not_valid], ignore_index=True)
valid_pw = valid_pw[~not_valid]

valid_pw.shape[0], bad_pw.shape[0]

(257, 725)

In [34]:
# 4.Must not contain the user's first or last name
valid_pw["name"] = valid_pw["username"].str.extract('(^[a-z]+)', expand=True)
valid_pw["surname"] = valid_pw["username"].str.extract('([a-z]+$)')

valid_pw.head()

,id,username,password,name,surname
0,1,vance.jennings,vanceRules888!,vance,jennings
1,2,consuelo.eaton,Mail_Pen%Scarlets.414,consuelo,eaton
4,5,araceli.wilder,Araceli}r3,araceli,wilder
6,7,evelyn.gay,`4:&iAt$'o~(,evelyn,gay
8,9,gladys.ward,=Wj1`i)xYYZ,gladys,ward


In [37]:
for i, row in valid_pw.iterrows():
    if row['name'] in row['password'].lower() or row['surname'] in row['password'].lower():
        print(row)
        valid_pw = valid_pw.drop(index=i)
        bad_pw = bad_pw.append(row, ignore_index=True)

id                       1
username    vance.jennings
password    vanceRules888!
name                 vance
surname           jennings
Name: 0, dtype: object
id                       5
username    araceli.wilder
password        Araceli}r3
name               araceli
surname             wilder
Name: 4, dtype: object
id                       12
username    milford.hubbard
password       Milford<3Tom
name                milford
surname             hubbard
Name: 11, dtype: object
id                      141
username      ronald.brooks
password    P1G_bT”_zBrooks
name                 ronald
surname              brooks
Name: 140, dtype: object
id                     150
username    raymundo.haley
password    HaleyComet333$
name              raymundo
surname              haley
Name: 149, dtype: object
id                    668
username    simon.miranda
password       SimonR0ck$
name                simon
surname           miranda
Name: 667, dtype: object
id                     750
username    i

In [39]:
bad_pw.head()

,id,username,password,name,surname
0,3,mitchel.perkins,Z00+1960,NaN,NaN
1,4,odessa.vaughan,D-rockyou,NaN,NaN
2,10,brant.zimmerman,L?4)OSB$r,NaN,NaN
3,17,domingo.dyer,VeOw{*p,NaN,NaN
4,18,martin.pacheco,MP1985???,NaN,NaN


In [40]:
bad_pass = round(bad_pw.shape[0]/df.shape[0],2)
bad_pass

0.75

In [42]:
email_list = bad_pw['username'].sort_values()
email_list

405           abdul.rowland
309            addie.cherry
372            adele.moreno
517            adeline.bush
279             adolfo.kane
337             adolfo.lara
16             ahmad.hopper
122              aida.combs
700           aisha.jenkins
199               al.dunlap
147            alana.franco
593         alberta.leblanc
521            alec.robbins
671    alejandra.stephenson
434         alejandro.burke
482        alejandro.nieves
205        alexander.thomas
400       alexandria.hinton
453       alexis.mccullough
93          alexis.reynolds
568          alfonso.weaver
151           alfonzo.johns
611          alisa.campbell
342             alisa.cohen
567             alison.neal
190          allan.marshall
142           alonzo.fowler
652           amado.bridges
88         amado.fitzgerald
592           amber.summers
               ...         
20              ursula.wood
280       valentin.castillo
596           valeria.curry
725          vance.jennings
319           vaness